In [4]:
import base64
import requests
import datetime
import pandas as pd
from serverside.client_cred import client_id,client_secret,playlist_id

In [5]:
client_id = client_id
client_secret = client_secret
playlist_id = playlist_id

In [6]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    playlist_id = None
    
    def __init__(self ,client_id,client_secret,playlist_id,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.playlist_id = playlist_id
        
    def get_token_data(self):
        return {
        'grant_type':'client_credentials',
                }
        
    def get_client_cred(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception('you must set client_id and client_token')
        client_cred = f"{client_id}:{client_secret}"
        client_cred_b64 = base64.b64encode(client_cred.encode())
        return client_cred_b64
    
    def get_token_headers(self): 
        client_cred_b64 = self.get_client_cred()
        return {
        'Authorization':f'Basic {client_cred_b64.decode()}',
                }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_headers()
        response = requests.post(token_url, data=token_data,headers=token_header)
        if response.status_code not in range(200,299):
            return False  
        token_response = response.json()
        access_token = token_response['access_token']
        now = datetime.datetime.now()
        expires_in = token_response['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires  = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_playlist_track_ids(self):
        header = {
            'Authorization':f'Bearer {self.access_token}',
        }
        endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}'
        r = requests.get(endpoint,headers=header).json()
        track_list = r['tracks']['items']
        n = len(track_list)
        track_ids = []
        for i in range(0,n-1):
            track_ids.append(track_list[i]['track']['id'])
        return track_ids
    
    def get_playlist_track_list(self):
        track_ids = self.get_playlist_track_ids()
        tracks_info = {}
        n = len(track_ids)
        for i in range (0,n):
            header = {
                'Authorization':f'Bearer {self.access_token}',
            }
            endpoint = f'https://api.spotify.com/v1/audio-features/{track_ids[i]}'
            r = requests.get(endpoint,headers=header).json()
            tracks_info[f'{i}'] = {     'track_id':f'{track_ids[i]}',
                                        'danceability':r['danceability'],
                                        'enegy':r['energy'],
                                        'loudness':r['loudness'],
                                        'speechiness':r['speechiness'],
                                        'acousticness':r['acousticness'],
                                        'instrumentalness':r['instrumentalness'],
                                        'liveness':r['liveness'],                        
                                        'tempo':r['tempo'],
                
            }
        return tracks_info

        
    

In [9]:
client = SpotifyAPI(client_id,client_secret,playlist_id)
client.perform_auth()


True

In [10]:
client.get_playlist_track_ids()
tracks_info = client.get_playlist_track_list()
df = pd.DataFrame(tracks_info).transpose()
df.to_csv('tracks.csv',index=False,header=True)